In [131]:
# adjusting antonio's code

In [132]:
# PATHS TO FILES
# Attempt 1 at basic code assigning a movie/ads spaces to a schedule

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [133]:
xp.init('C:/xpressmp/bin/xpauth.xpr')

In [134]:
ch_A_schedule.head()

,Unnamed: 0,children_baseline_view_count,adults_baseline_view_count,retirees_baseline_view_count,prime_time_factor
0,2024-10-01 07:00:00,0.020000,0.022208,0.003218,1.0
1,2024-10-01 07:05:00,0.019931,0.022218,0.003367,1.0
2,2024-10-01 07:10:00,0.019724,0.022247,0.003522,1.0
3,2024-10-01 07:15:00,0.019385,0.022295,0.003684,1.0
4,2024-10-01 07:20:00,0.018919,0.022362,0.003851,1.0


In [135]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)


In [136]:
ch_A_schedule.head()


,children_baseline_view_count,adults_baseline_view_count,retirees_baseline_view_count,prime_time_factor
Date,,,,
2024-10-01 07:00:00,0.020000,0.022208,0.003218,1.0
2024-10-01 07:05:00,0.019931,0.022218,0.003367,1.0
2024-10-01 07:10:00,0.019724,0.022247,0.003522,1.0
2024-10-01 07:15:00,0.019385,0.022295,0.003684,1.0
2024-10-01 07:20:00,0.018919,0.022362,0.003851,1.0


In [137]:
# Slot duration 30 minutes
slot_duration = 30  # minutes
movies_df['slots_needed'] = (movies_df['runtime_with_ads'] / slot_duration).apply(lambda x: int(x)).astype(int)


In [138]:
# Define the broadcasting date

k = 7
start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
test_range = pd.date_range(start_date, periods = k)

broadcast_date = []
for i in test_range:
    broadcast_date.append(i)
for i in broadcast_date:
    broadcast_start = i.replace(hour=7, minute=0)
    broadcast_end = i.replace(hour=23, minute=30)
# print(broadcast_date)
    

# Generate all time slots
 
time_slots = []
current_time = broadcast_start
while current_time <= broadcast_end:
    time_slots.append(current_time)
    current_time += timedelta(minutes=slot_duration)

# Create mappings between time slots and indices
time_to_index = {t: idx for idx, t in enumerate(time_slots)}
index_to_time = {idx: t for idx, t in enumerate(time_slots)}


In [139]:
ch_A_schedule.head(10)

,children_baseline_view_count,adults_baseline_view_count,retirees_baseline_view_count,prime_time_factor
Date,,,,
2024-10-01 07:00:00,0.020000,0.022208,0.003218,1.0
2024-10-01 07:05:00,0.019931,0.022218,0.003367,1.0
2024-10-01 07:10:00,0.019724,0.022247,0.003522,1.0
2024-10-01 07:15:00,0.019385,0.022295,0.003684,1.0
2024-10-01 07:20:00,0.018919,0.022362,0.003851,1.0
2024-10-01 07:25:00,0.018337,0.022448,0.004025,1.0
2024-10-01 07:30:00,0.017650,0.022553,0.004205,1.0
2024-10-01 07:35:00,0.016871,0.022678,0.004392,1.0
2024-10-01 07:40:00,0.016015,0.022821,0.004585,1.0


In [140]:
# Set 'Date' as index
#ch_A_schedule.set_index('Date', inplace=True)

# Resample to 30-minute intervals (use mean of groups)
ch_A_schedule_30min = ch_A_schedule.resample('30T').mean(numeric_only = True).reset_index()



C:\Users\Elie\AppData\Local\Temp\ipykernel_12392\290806634.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_A_schedule_30min = ch_A_schedule.resample('30T').mean(numeric_only = True).reset_index()


In [141]:
# Create a dictionary to hold baseline viewership per time slot
baseline_viewership = {}

for idx, row in ch_A_schedule_30min.iterrows():
    time_slot_time = row['Date'].time()
    baseline_viewership[time_slot_time] = {
        'children': row['children_baseline_view_count'],
        'adults': row['adults_baseline_view_count'],
        'retirees': row['retirees_baseline_view_count'],
        'prime_time_factor': row.get('prime_time_factor', 1)
    }


In [142]:
# Initialize the problem
prob = xp.problem(name="Movie_Scheduling_Problem")


In [143]:
# Check for duplicate movie titles
duplicate_titles = movies_df[movies_df.duplicated(subset=['title'], keep=False)]
if not duplicate_titles.empty:
    print("Duplicate movie titles found:")
    print(duplicate_titles['title'])
else:
    print("No duplicate movie titles found.")


Duplicate movie titles found:
4                 The Avengers
17                     Titanic
76               The Lion King
105       Beauty and the Beast
149        Alice in Wonderland
                 ...          
5748              Midnight Sun
5761                The Island
5773            The Shaggy Dog
5855    Fun with Dick and Jane
5879        The Perfect Weapon
Name: title, Length: 258, dtype: object


In [144]:
movies_df.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres,n_ad_breaks,runtime_with_ads,scaled_popularity,children_scaled_popularity,adults_scaled_popularity,retirees_scaled_popularity,slots_needed
0,Inception,8.364,34495,2010-07-15,825532764,148,160000000,83.952,"['Action', 'Science Fiction', 'Adventure']",5,180.0,0.972027,0.777622,0.972027,0.388811,6
1,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,140.241,"['Adventure', 'Drama', 'Science Fiction']",6,210.0,1.000000,0.600000,1.000000,0.600000,7
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,130.643,"['Drama', 'Action', 'Crime', 'Thriller']",6,180.0,1.000000,0.600000,1.000000,0.800000,6
3,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,79.932,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",6,180.0,0.958199,0.766559,0.958199,0.383280,6
4,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,98.082,"['Science Fiction', 'Action', 'Adventure']",5,180.0,1.000000,0.800000,1.000000,0.400000,6


In [145]:
# Create variables x_{mt} (which movie m is shown at time t)
used_movies_df = pd.DataFrame()
x = {} 
for m_idx, m_row in movies_df.iterrows():
    m = m_row['title'] # gives just the titles
    m_id = f"{m.replace(' ', '_')}_{m_idx}" 
    for t in range(len(time_slots)):
        var_name = f"x_{m_id}_{t}"
        x[(m_idx, t)] = xp.var(vartype=xp.binary, name=var_name)
        prob.addVariable(x[(m_idx, t)])














    
    # find which titles are used in dataframe, find their index, get rid of the whole row so the movie can't be used again
    # take the removed title and add to a df of "used movies" so we can keep track of which one have and  have not been used
    if x[(m_idx, t)] > 0.5:
        movie_used = movies_df.drop(movies_df.where(movies_df['title'] == m), axis = 0, inplace = True)
        used_movies_df.append(movie_used)
















# Create variables y_m (if movie m is shown, binary) 
y = {}
for m_idx, m_row in movies_df.iterrows():
    m = m_row['title']
    m_id = f"{m.replace(' ', '_')}_{m_idx}"
    var_name = f"y_{m_id}"
    y[m_idx] = xp.var(vartype=xp.binary, name=var_name)
    prob.addVariable(y[m_idx])

# Create variables s_m (start time of movie m)
s = {}
for m_idx, m_row in movies_df.iterrows():
    m = m_row['title']
    m_id = f"{m.replace(' ', '_')}_{m_idx}"
    var_name = f"s_{m_id}"
    s[m_idx] = xp.var(vartype=xp.integer, name=var_name, lb=0, ub=len(time_slots)-1)
    prob.addVariable(s[m_idx])

# Create variables e_m (end time of movie m)
e = {}
for m_idx, m_row in movies_df.iterrows():
    m = m_row['title']
    m_id = f"{m.replace(' ', '_')}_{m_idx}"
    var_name = f"e_{m_id}"
    e[m_idx] = xp.var(vartype=xp.integer, name=var_name, lb=0, ub=len(time_slots)-1)
    prob.addVariable(e[m_idx])


ModelError: Invalid constraint: cannot enforce f(x) != 0, f(x) > 0, or f(x) < 0

In [ ]:
# Map baseline viewership to time slot indices
baseline_viewership = {}
for idx, row in ch_A_schedule_30min.iterrows():
    time_slot_datetime = row['Date']
    # Find the corresponding time slot index
    time_slot_index = time_to_index.get(time_slot_datetime, None)
    if time_slot_index is not None:
        baseline_viewership[time_slot_index] = {
            'children': row['children_baseline_view_count'],
            'adults': row['adults_baseline_view_count'],
            'retirees': row['retirees_baseline_view_count'],
            'prime_time_factor': row.get('prime_time_factor', 1)
        }


In [ ]:
print("Baseline Viewership Data:")
for key, value in list(baseline_viewership.items())[:5]:  # Print first 5 entries
    print(f"Time Slot {key}: {value}")


In [ ]:
# Total population (adjust as needed)
total_population = 1_000_000 

# Compute base viewership
base_viewership = {}
for m_idx, m_row in movies_df.iterrows():
    m = m_row['title']
    m_id = f"{m.replace(' ', '_')}_{m_idx}"
    # Scaled popularity for the movie
    scaled_popularity = {
        'children': m_row['children_scaled_popularity'],
        'adults': m_row['adults_scaled_popularity'],
        'retirees': m_row['retirees_scaled_popularity']
    }
    for t in range(len(time_slots)):
        # Baseline viewership for the time slot
        baseline = baseline_viewership.get(t, {
            'children': 0,
            'adults': 0,
            'retirees': 0,
            'prime_time_factor': 1
        })
        # Calculate base viewership for this movie and time slot
        viewership = 0
        for group in ['children', 'adults', 'retirees']:
            group_viewership = (
                baseline[group] *
                scaled_popularity[group] *
                baseline['prime_time_factor'] *
                total_population
            )
            viewership += group_viewership
        # Store the calculated viewership
        base_viewership[(m_idx, t)] = viewership



In [ ]:
# PRINT EXAMPLE OF BASE VIEWERSHIP. CONTAINS VIEWERSHIP PER MOVIE PER TIME SLOT (sums all groups)
print("Sample of base_viewership:")
sample_items = list(base_viewership.items())[:5]
for key, value in sample_items:
    print(f"{key}: {value}")


In [ ]:
# Create the objective function expression
objective_terms = []
for m_idx, m_row in movies_df.iterrows():
    for t in range(len(time_slots)):
        var = x[(m_idx, t)]
        coeff = base_viewership.get((m_idx, t), 0)
        if coeff != 0:
            objective_terms.append(coeff * var)

# Set the objective function
prob.setObjective(xp.Sum(objective_terms), sense=xp.maximize)


In [ ]:
# Constraint 1: Time Slot Occupancy Constraint
for t in range(len(time_slots)):
    prob.addConstraint(
        xp.Sum(x[(m_idx, t)] for m_idx in movies_df.index) == 1
    )


In [ ]:
# Constraint 2: Movie Duration Constraint
for m_idx, m_row in movies_df.iterrows():
    d_m = m_row['slots_needed']
    duration = xp.Sum(x[(m_idx, t)] for t in range(len(time_slots)))
    prob.addConstraint(duration == y[m_idx] * d_m)

In [ ]:
#Constraint 3: End Time Definition Constraint
for m_idx in movies_df.index:
    for t in range(len(time_slots)):
        end_time_constraint = e[m_idx] >= t * x[(m_idx, t)]
        prob.addConstraint(end_time_constraint)

In [ ]:
# Constraint 4: Movie Duration Relationship Constraint
for m_idx, m_row in movies_df.iterrows():
    d_m = m_row['slots_needed']
    duration_relationship = e[m_idx] - s[m_idx] + 1 == y[m_idx] * d_m
    prob.addConstraint(duration_relationship)

In [ ]:
# Define Big-M
M = len(time_slots)

# Constraint 5: Start Time Constraint
for m_idx in movies_df.index:
    for t in range(len(time_slots)):
        start_time_constraint = s[m_idx] <= t + M * (1 - x[(m_idx, t)])
        prob.addConstraint(start_time_constraint)

In [ ]:
# Constraint 6: Contiguity Constraint
for m_idx, m_row in movies_df.iterrows():
    d_m = m_row['slots_needed']
    for t in range(len(time_slots)):
        # Enforce x_{mt} == 1 if t is within [s_m, s_m + d_m -1]
        # Using Big-M constraints
        
        # x_{mt} >= y_m - (s_m > t + d_m -1)
        # Since we cannot use logical operators, use linear constraints
        
        # If t >= s_m and t < s_m + d_m, then x_{mt} >= y_m
        # Otherwise, x_{mt} >= 0 (which is already ensured since x is binary)
        
        # Implement the following:
        # x_{mt} >= y_m + (t - s_m) / M - (t - s_m - d_m + 1) / M - 1
        # This is a linear approximation and may not strictly enforce contiguity
        # Hence, it's better to use integer variables and enforce exact relations
        
        # Instead, we'll use the relationship defined in Constraint 4 to indirectly enforce contiguity
        pass  # Already handled by Constraints 2 and 4

In [ ]:
# Constraint 7: Broadcasting Time Limit Constraint
T_end = len(time_slots) - 1  # Last time slot index

for m_idx, m_row in movies_df.iterrows():
    d_m = m_row['slots_needed']
    broadcasting_limit = s[m_idx] + d_m - 1 <= T_end
    prob.addConstraint(broadcasting_limit)

In [ ]:
# Set solver options if needed (optional)
# For example, to set the time limit or verbosity
# xp.control.outputlog = 1  # Enable solver output
# xp.control.maxtime = 600  # Set maximum solving time to 600 seconds

# Solve the problem
prob.solve()

# Retrieve the solver status
status = prob.getProbStatus()
print(f"Solver Status: {status}")

In [ ]:
# Function to convert time slot index to actual time
def get_time(slot_index):
    return time_slots[slot_index].time()

# Initialize a list to store scheduled movies
scheduled_movies = []

for m_idx, m_row in movies_df.iterrows():
    # Retrieve the solution value for y[m_idx]
    y_value = prob.getSolution(y[m_idx])

    if y_value > 0.5:  # Movie is scheduled
        # Retrieve solution values for s[m_idx] and e[m_idx]
        start_slot = int(prob.getSolution(s[m_idx]))
        end_slot = int(prob.getSolution(e[m_idx]))
        
        # Convert slot indices to actual times
        start_time = get_time(start_slot)
        end_time = get_time(end_slot)

        # Append the scheduled movie details
        scheduled_movies.append({
            'Movie Index': m_idx,
            'Movie Title': m_row['title'],
            'Start Slot': start_slot,
            'Start Time': start_time.strftime('%H:%M'),
            'End Slot': end_slot,
            'End Time': end_time.strftime('%H:%M')
        })

# Convert to DataFrame for better visualization
for i in range(k):
    schedule_df = pd.DataFrame(scheduled_movies)
    schedule_df.sort_values(['Start Slot'], axis = 0, inplace = True)

    print("Scheduled Movies:", i)
    print(schedule_df)

In [ ]:
# Initialize a list to store time slot assignments
# time_slot_assignments = []

# for t in range(len(time_slots)):
#     for m_idx in movies_df.index:
#         if prob.getSolution(x[(m_idx, t)]) > 1:
#             movie_title = movies_df.loc[m_idx, 'title']
#             start_slot = int(prob.getSolution(s[m_idx]))
#             start_time = get_time(start_slot)
#             end_slot = int(prob.getSolution(e[m_idx]))
#             end_time = get_time(end_slot)
#             time_slot_assignments.append({
#                 'Time Slot Index': t,
#                 'Time': time_slots[t].strftime('%H:%M'),
#                 'Movie Title': movie_title,
#                 'Start Time': start_time.strftime('%H:%M'),
#                 'End Time': end_time.strftime('%H:%M')
#             })
#             break  # Since only one movie per time slot

# # Convert to DataFrame
# schedule_mapping_df = pd.DataFrame(time_slot_assignments)

# print("\nTime Slot Assignments:")
# print(schedule_mapping_df)